In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from pytorch3d.io import load_obj
from pytorch3d.renderer import (
    MeshRenderer, MeshRasterizer, FoVPerspectiveCameras, RasterizationSettings, SoftSilhouetteShader
)
from pytorch3d.structures import Meshes
from scipy.spatial.transform import Rotation

In [ ]:
from pytorch3d.io import load_objs_as_meshes

device = torch.device("cuda:0")
meshes = load_objs_as_meshes(['data/cow_mesh/cow.obj'], device=device)

In [19]:
from pytorch3d.renderer import look_at_view_transform, PointLights, SoftPhongShader
from torch import Tensor

torch.no_grad()

fov = 60
sensor_size = 300

def euler(x, y, z):
    angles = [x, y, z]
    angles_rad = [np.radians(angle) for angle in angles]
    return Tensor(Rotation.from_euler('xyz', angles_rad).as_matrix()).unsqueeze(0)

# Initialize a camera.
# With world coordinates +Y up, +X left and +Z in, the front of the cow is facing the -Z direction. 
# So we move the camera by 180 in the azimuth direction so it is facing the front of the cow. 

# this is w2c!
R, T = look_at_view_transform(2.7, 0, 0) 
cameras = FoVPerspectiveCameras(device=device, R=R, T=T, fov=fov)

raster_settings = RasterizationSettings(
    image_size=sensor_size,
    blur_radius=0.0,
    faces_per_pixel=50,
)

lights = PointLights(device=device, location=[[0.0, 0.0, -3.0]])

renderer = MeshRenderer(
    rasterizer=MeshRasterizer(cameras=cameras, raster_settings=raster_settings),
    shader=SoftPhongShader(device=device, cameras=cameras, lights=lights)
)

render_og = renderer(meshes, cameras=cameras).cpu().numpy()
render = render_og[0]

def rr_meshes(meshes, batch_idx=0):
    # extract verts and faces from idx-th mesh in batch    
    verts = meshes.verts_list()[batch_idx].cpu()
    faces = meshes.faces_list()[batch_idx].cpu()
    vertex_normals = meshes.verts_normals_list()[batch_idx].cpu()
    
    tex = meshes.textures.maps_padded()[batch_idx].cpu().numpy()
    uv = meshes.textures.verts_uvs_padded()[0].cpu().numpy()

    return rr.Mesh3D(
        vertex_positions=verts,
        triangle_indices=faces,
        vertex_normals=vertex_normals,
    )

focal_length = int(sensor_size / (2 * np.tan(fov * np.pi / 360)))

import rerun as rr
rr.init(spawn=True, application_id='meshvis')

rr.log('/', rr.ViewCoordinates.LUF, static=True)
rr.log('/', rr.Transform3D())

rr.log("mesh", rr_meshes(meshes))
rr.log('cam', rr.Pinhole(width=sensor_size, height=sensor_size, focal_length=focal_length, camera_xyz=rr.ViewCoordinates.RIGHT_HAND_Y_UP))
rr.log('cam', rr.Transform3D(translation=T.cpu(), mat3x3=R.cpu()))
rr.log('cam', rr.Image(render))

print(T)
print(R)

tensor([[-0.0000, -0.0000, 2.7000]])
tensor([[[-1.,  0.,  0.],
         [ 0.,  1.,  0.],
         [ 0.,  0., -1.]]])


[2024-08-20T14:59:25Z INFO  re_sdk::spawn] A process is already listening at this address. Assuming it's a Rerun Viewer. addr=0.0.0.0:9876
[2024-08-20T14:59:25Z INFO  re_sdk_comms::server] New SDK client connected from: 127.0.0.1:46268


In [38]:
from pytorch3d.transforms import Transform3d

t = Transform3d()

In [ ]:
cameras